In [ ]:
from text_processing.runner import TextProcessingRunner

from summarizer.seq2seq.model import Seq2SeqModel
from summarizer.seq2seq.training import Seq2SeqTraining

from IPython.display import clear_output

from configuration import mkdir_if_not_exists

rootdir_data = './data/'
data_name = 'annota-title-fulltext'
vec_name = 'en/300'
model_nickname = 'slurpee'
save_path = {
    'model': './models/{}_{}/model'.format(data_name, model_nickname),
    'variables': './models/{}_{}/variables'.format(data_name, model_nickname),
    'logs': './models/{}_{}/example_logs.txt'.format(data_name, model_nickname),
}
configurations = { 
    data_name,
    'vectors/{}'.format(vec_name),
    'multilayer',
    'eco',
}
linecount = None

In [ ]:
def make_missing_dirs(data_name, vec_name, model_nickname):
    mkdir_if_not_exists('./data/embeddings/{}'.format(data_name))
    mkdir_if_not_exists('./data/embeddings/{}/{}'.format(data_name, vec_name.split('/')[0]))
    mkdir_if_not_exists('./data/embeddings/{}/{}/{}'.format(data_name, vec_name.split('/')[0], vec_name.split('/')[1]))
    mkdir_if_not_exists('./data/batches/{}'.format(data_name))
    mkdir_if_not_exists('./models/{}_{}'.format(data_name, model_nickname))

make_missing_dirs(data_name=data_name,
                  vec_name=vec_name,
                  model_nickname=model_nickname)

In [ ]:
# Process data
text_processing = TextProcessingRunner(configurations=configurations, 
                                       rootdir=rootdir_data, 
                                       linecount=linecount, 
                                       checkpoint=20)
text_processing.run()

In [ ]:
# Init model
model = Seq2SeqModel(configurations=configurations, 
                     rootdir=rootdir_data)
model.initialize()
trainer = Seq2SeqTraining(model)

In [ ]:
# Load model and variables from files
model.load(save_path['model'])
trainer.load(save_path['variables'])

In [ ]:
# Train
trainer.run(print_after_steps=500,
            output_file=save_path['logs'])

In [ ]:
# Check epochs
print('Epochs: {}'.format(model.train_iter.epochs))
# Check graphs of trained model
trainer.print_graphs()

In [ ]:
# Save model and variables for further use
model.save(save_path['model'])
trainer.save(save_path['variables'])